## Семинар 8: "Современные модели для NLP"

ФИО: Сульженко Родион Вадимович

### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~250MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [1]:
import torch
!pip install transformers
from transformers import *

In [2]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-cased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [3]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 3446, 1110, 1199, 3087, 1106, 4035, 13775, 102]


In [4]:
tokenizer.decode(input_ids)

'[CLS] Here is some text to encode [SEP]'

In [5]:
input_ids[4] = tokenizer.mask_token_id
tokenizer.decode(input_ids)

'[CLS] Here is some [MASK] to encode [SEP]'

In [6]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [7]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [8]:
tokenizer.decode(new_ids.numpy()[0, :].tolist())

'. here is some way to encode.'

In [9]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

In [10]:
input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)
print(input_ids)

[101, 1130, 170, 19196, 4006, 117, 7482, 2751, 170, 17804, 1104, 8362, 23941, 1116, 1690, 1107, 170, 6456, 117, 2331, 25731, 1775, 1643, 21425, 1181, 4524, 117, 1107, 1103, 19505, 5249, 119, 2431, 1167, 11567, 1106, 1103, 6962, 1108, 1103, 1864, 1115, 1103, 8362, 23941, 1116, 2910, 3264, 1483, 119, 102]


##### Выбор самого вероятного слова

In [11]:
from tqdm.notebook import tqdm

input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)

max_len = 1000
for i in tqdm(range(max_len)):
    ### [idx, idx, ... , idx, SEP]
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    ### [idx, idx, ... , idx, MASK, SEP]
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0)
    with torch.no_grad():
         res = model(input_batch)[0]
    prob = torch.nn.functional.softmax(res, dim=-1)
    new_idx = prob.max(-1)[1][0].numpy()
    input_ids[len(input_ids) - 2] = new_idx[len(input_ids[i:]) - 2] ## mask -> new_idx
    ### [idx, idx, ... , idx, new_idx, SEP]
tokenizer.decode(input_ids)

'[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. ” ” ” ”. ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ” ”

In [12]:
from tqdm.notebook import tqdm

input_ids = tokenizer.encode(GPT_TEXTS[1], add_special_tokens=True)

max_len = 1000
for i in tqdm(range(max_len)):
    ### [idx, idx, ... , idx, SEP]
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    ### [idx, idx, ... , idx, MASK, SEP]
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0)
    with torch.no_grad():
         res = model(input_batch)[0]
    prob = torch.nn.functional.softmax(res, dim=-1)
    new_idx = prob.max(-1)[1][0].numpy()
    input_ids[len(input_ids) - 2] = new_idx[len(input_ids[i:]) - 2] ## mask -> new_idx
    ### [idx, idx, ... , idx, new_idx, SEP]
tokenizer.decode(input_ids)

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. " ". " ". " ". " ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". ". 

##### Семплирование

In [13]:
import numpy as np
input_ids = tokenizer.encode(GPT_TEXTS[0], add_special_tokens=True)

for i in tqdm(range(max_len)):
    ### [idx, idx, ... , idx, SEP]
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    ### [idx, idx, ... , idx, MASK, SEP]
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0)
    with torch.no_grad():
        res = model(input_batch)[0]
    prob = torch.nn.functional.softmax(res, dim=-1)
    # Без topk как-то совсем плохо
    new_idx = torch.topk(prob, 1000)[1][0][:, np.random.randint(0, 1000)].numpy()
    input_ids[len(input_ids) - 2] = new_idx[len(input_ids[i:]) - 2]
    ### [idx, idx, ... , idx, new_idx, SEP]
    
tokenizer.decode(input_ids)

"[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. Marieel speaks brightly but gradually oversimate audiences demonstrating this dialogue strikingl theme playing greatly resembling : setting fields opening each monster cliff godain glimpse imminent blackness them concealed beautiful waters lush levels realm } behind altar shell positions began sailing when retreating quickly position fore shadows ourselves looking then clearer others greatly also with blades immediately occur dynamic frames away finally upper cloud lair exterior images made once beneath slab rising mirror forests preserve what voices ) nearbyward houses added vertex connections come sheet distance removed fast noise connections nothing earlier ; daylight arrival begin 1976 recommended silver enclosure operations schedule became total t

In [14]:
input_ids = tokenizer.encode(GPT_TEXTS[1], add_special_tokens=True)

for i in tqdm(range(max_len)):
    ### [idx, idx, ... , idx, SEP]
    input_ids.insert(len(input_ids) - 1, tokenizer.mask_token_id)
    ### [idx, idx, ... , idx, MASK, SEP]
    input_batch = torch.tensor(input_ids[i:]).to(torch.long).unsqueeze(0)
    with torch.no_grad():
        res = model(input_batch)[0]
    prob = torch.nn.functional.softmax(res, dim=-1)
    new_idx = torch.topk(prob, 1000)[1][0][:, np.random.randint(0, 1000)].numpy()
    input_ids[len(input_ids) - 2] = new_idx[len(input_ids[i:]) - 2]
    ### [idx, idx, ... , idx, new_idx, SEP]
    
tokenizer.decode(input_ids)

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. ， physics c Siueckerin § IV ≈ diagramplane file 287 } 229 39 280 13 9 wpo 127 " Conway notes29Q ` 321 225 e 1971 1792 149º 3tamn 230ᵃ 68 bit 5 Vdah 77ah ps\'slab n J 52 ] 104 grams dust δ ₑᵘত 19246 fragments 1930 illustrations86 billion⁴41btER ₑ 112₅RNAᵇ 111 # 1600 122xy³ʳ55 hampushoffius 171 177 BChus 188 1648 1640 24 holes trick 57 2018 12340 played 43lts 06 79 lb 1878 3rd phase 223 shots 2011 69 wounds 173 fails 128 students 1968 questions check 228gslevysars problem simulation or functions either 34 vary assessments 46 validity 600X million spheres measurements – 93 boards 154 countries C 87 objectives periods courses up offers 34 index songs facilitieses 85 practices rating horizon episodes a • scenario activities criteria evaluating universities ijefa concept4 members share items AS 7mes within instances disciplines assessment network API instrument across 

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: